# Bài toán thủy lực 2D

## Sai phân hóa hệ phương trình Reynolds
Viết lại hệ phương trình Reynolds

$$
\frac{{\partial u}}{{\partial t}} + u\frac{{\partial u}}{{\partial x}} + v\frac{{\partial u}}{{\partial y}} - fv =  - g\frac{{\partial \zeta }}{{\partial x}} + \frac{{{\tau _{sx}}}}{{\rho \left( {h + \zeta } \right)}} - Ku\frac{{\sqrt {{u^2} + {v^2}} }}{{h + \zeta }} + A{\nabla ^2}u$$

$$
\frac{{\partial v}}{{\partial t}} + u\frac{{\partial u}}{{\partial x}} + v\frac{{\partial u}}{{\partial y}} + fu =  - g\frac{{\partial \zeta }}{{\partial y}} + \frac{{{\tau _{sy}}}}{{\rho \left( {h + \zeta } \right)}} - Kv\frac{{\sqrt {{u^2} + {v^2}} }}{{h + \zeta }} + A{\nabla ^2}v$$

$$
\frac{{\partial \zeta }}{{\partial t}} + \frac{\partial }{{\partial x}}\left[ {\left( {h + \zeta } \right)u} \right] + \frac{\partial }{{\partial y}}\left[ {\left( {h + \zeta } \right)v} \right] = 0$$